# Modeling How to Beat the Streak

In 1941, Joe DiMaggio recorded hits in 56 consecutive games of Major League Baseball. This record has stood for 80 years and is seen as one of the most unlikely to be broken. In 2001, MLB started a fantasy game called Beat the Streak, challenging fans to simply pick, among all player, one who would get a hit on a given day. And then do that for 57 consecutive days to symbolically beat DiMaggio's streak. In 20 years nobody has won but a few have come as close as 51.

In this notebook I aim to take a statistical approach to determining these daily picks in order to give myself the best possible chance I can to win this competition and, with it, the $5.6 million prize.

My first step will be to gather the relevant data and assemble it in a useful way.

In [1]:
import os
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import statsapi
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')


import pybaseball

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, plot_confusion_matrix, precision_score, plot_roc_curve
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier


import xgboost as xgb

import category_encoders as ce

from dateutil import tz

from time import sleep

import pickle

pybaseball.statcast allows me to gather details from of every MLB game since 2017 down to the individual pitch level. As this alone is a lot of data I'll save it as I gather it so I don't have to rerun this part. 

As I take it in I'll transform it to save on memory since the dataset is quite large and will take a while to run.

In [2]:
def statcast_importer(year = datetime.now().year):
    start = datetime(year, 1, 1)
    if year == datetime.now().year:
        end = datetime.now()-timedelta(weeks=1)
    else:
        end = datetime(year,12, 31)
    df = pybaseball.statcast(str(start.year) + '-' + '{:0>2}'.format(str(start.month)) + '-' + '{:0>2}'.format(str(start.day)), str(end.year) + '-' + '{:0>2}'.format(str(end.month)) + '-' + '{:0>2}'.format(str(end.day)))
    df.to_csv('data/untouched_' + str(year) + '_statcast_pbp.csv')

    num_columns = ['release_speed', 'batter', 'pitcher', 'zone', 'hit_location', 'balls', 'strikes', 'game_year', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'fielder_2', 'hit_distance_sc', 'release_spin_rate', 'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'woba_value', 'woba_denom', 'babip_value', 'at_bat_number', 'pitch_number', 'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score']

    num_transformer = Pipeline(steps=[
    ('num_imputer', SimpleImputer(strategy='median'))
    ])
    preprocessor = ColumnTransformer(transformers=[('num', num_transformer, num_columns)])
    clf = Pipeline(steps=[('preprocessor', preprocessor)])

    t = clf.fit_transform(df)

    df[num_columns] = t

#   drop spring training, playoff games

    df = df[df['game_type'] == 'R']
    print('Non-regular season games dropped')


    df.to_csv('data/'+str(year)+'_statcast_pbp.csv')
    return(df)

In [3]:
# establish definitions for the events column for what ends a plate appearance ending event and a hit
# the statcast includes data for every pitch which needs to be pared down to determine whether a player got a hit
pa_ending_events = np.array(['field_out',
                        'strikeout',
                        'single',
                        'walk',
                        'double',
                        'home_run',
                        'force_out',
                        'grounded_into_double_play',
                        'hit_by_pitch',
                        'field_error',
                        'sac_fly',
                        'triple',
                        'sac_bunt',
                        'fielders_choice',
                        'double_play',
                        'fielders_choice_out',
                        'strikeout_double_play',
                        'catcher_interf',
                        'sac_fly_double_play',
                        'triple_play',
                        'sac_bunt_double_play'])


hit_events = np.array(['single',
            'double',
            'home_run',
            'triple'])

american_league_teams = np.array(['Boston Red Sox', 'Houston Astros', 'Chicago White Sox', 'Tampa Bay Rays', 'Oakland Athletics', 'Seattle Mariners', 'New York Yankees', 'Toronto Blue Jays', 'Los Angeles Angels', 'Cleveland Indians', 'Detroit Tigers', 'Kansas City Royals', 'Minnesota Twins', 'Texas Rangers', 'Baltimore Orioles'])


num_columns = ['release_speed', 'batter', 'pitcher', 'zone', 'hit_location', 'balls', 'strikes', 'game_year', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'fielder_2', 'hit_distance_sc', 'release_spin_rate', 'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'woba_value', 'woba_denom', 'babip_value', 'at_bat_number', 'pitch_number', 'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score']

column_types = [np.int8,
np.int32,
np.int32,
np.int8,
np.int8,
np.int8,
np.int8,
np.int16,
np.int32,
np.int32,
np.int32,
np.int32,
np.int32,
np.int32,
np.int16,
np.int16,
np.int32,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int8,
np.int8,
np.int8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8]

type_dict = dict(zip(num_columns, column_types))

This explicity sets types to the numeric features in order to save on memory.

In [4]:
def set_types(df, type_dict):
    '''Explicitly names datatypes in order to speed up preprocessing of a large dataset'''

    df['game_date'] = pd.to_datetime(df['game_date'], format='%Y-%m-%d')
    print('datetime done')
    df['events'] = df['events'].apply(lambda x: 1 if x in hit_events else (0 if x in pa_ending_events else -1)).astype(np.int8)
    print('events done')
    df['batter_righty'] = df['stand'].apply(lambda x: 1 if x == 'R' else 0).astype(np.int8)
    print('stand done')
    df['pitcher_righty'] = df['p_throws'].apply(lambda x: 1 if x == 'R' else 0).astype(np.int8)
    print('pitcher done')
    df['home'] = df['inning_topbot'].apply(lambda x: 1 if x == 'Bot' else 0).astype(np.int8)
    print('inning done')

    for key, value in type_dict.items():
        df[key] = df[key].astype(value)

    df = df.select_dtypes(exclude=['object'])
    for col in df.columns:
        if col.startswith('Unnamed:'):
            df = df.drop([col], axis=1)
    return (df)

Time zone conversion based on venue name (each is unique in Parks.csv) in order to pass the proper times to the Visual Crossing weather api

In [5]:
def convert_UTC_to_local(row):
    """Takes in a row containing a baseball game's location and UTC start time"""

    #     Reads in latitude, longitude, and altitude for known venues
    venue_coords = pd.read_csv('Parks.csv')

    venue_name = row['venue_name']
    game_datetime = row['game_datetime']

    #     get timezone object for UTC
    from_zone = tz.gettz('UTC')

    try:
        city = venue_coords[venue_coords['NAME'] == row['venue_name']]['CITY'].iloc[0]
        lat = venue_coords[venue_coords['NAME'] == row['venue_name']]['Latitude'].iloc[0]
        lon = venue_coords[venue_coords['NAME'] == row['venue_name']]['Longitude'].iloc[0]
        alt = venue_coords[venue_coords['NAME'] == row['venue_name']]['Altitude'].iloc[0]
    except NameError:
        #         Need to add new venue to Parks.csv
        raise IndexError(venue, index)
    if city == 'Tokyo':
        to_zone = tz.gettz('Asia/Tokyo')
    elif city == 'London':
        to_zone = tz.gettz('Europe/London')
    elif city in ['San Francisco',
                  'Oakland',
                  'Phoenix',
                  'Seattle',
                  'Los Angeles',
                  'San Diego',
                  'Anaheim']:
        to_zone = tz.gettz('America/Los_Angeles')
    elif city == 'Denver':
        to_zone = tz.gettz('America/Denver')
    elif city in ['Minneapolis',
                  'Milwaukee',
                  'Chicago',
                  'St. Louis',
                  'Arlington',
                  'Kansas City',
                  'Houston',
                  'Monterrey',
                  'Omaha',
                  'Dyersville']:
        to_zone = tz.gettz('America/Chicago')
    elif city in ['Buffalo',
                  'Detroit',
                  'Cincinnati',
                  'Pittsburgh',
                  'Tampa Bay',
                  'Philadelphia',
                  'Atlanta',
                  'New York',
                  'Washington',
                  'Cleveland',
                  'Miami',
                  'Boston',
                  'Baltimore',
                  'Toronto',
                  'Williamsport',
                  'Dunedin',
                  'St. Petersburg',
                  'Fort Bragg']:
        to_zone = tz.gettz('America/New_York')
    elif city == "San Juan":
        to_zone = tz.gettz('America/Puerto_Rico')
    else:
        raise NameError(venue_name, city)
    utc = datetime.strptime(game_datetime, '%Y-%m-%dT%H:%M:%SZ')
    #     Convert start time from
    utc = utc.replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

In [6]:
api_counter = 0

Set what portion of the historical statcast data we want to study. We'll start the collection in the 2015 season since that's the first one with launch angle and speed information.

In [7]:
for year in [2015, 2016, 2017, 2018, 2019, 2020, 2021]:
    statcast_importer(year)

This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 211/211 [00:48<00:00,  4.34it/s]


Non-regular season games dropped
This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 214/214 [00:46<00:00,  4.63it/s]


Non-regular season games dropped
This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


...

Non-regular season games dropped
This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 97/97 [00:22<00:00,  4.40it/s]


Non-regular season games dropped
This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 246/246 [00:43<00:00,  5.68it/s]


Non-regular season games dropped


In [8]:
df = set_types(pd.read_csv('data/2021_statcast_pbp.csv'), type_dict)

df = df[df['game_date']<= datetime(2021,6,30)]

df = pd.concat([set_types(pd.read_csv('data/2017_statcast_pbp.csv'), type_dict),
                set_types(pd.read_csv('data/2018_statcast_pbp.csv'), type_dict),
                set_types(pd.read_csv('data/2019_statcast_pbp.csv'), type_dict),
                set_types(pd.read_csv('data/2020_statcast_pbp.csv'), type_dict),
                set_types(pd.read_csv('data/2021_statcast_pbp.csv'), type_dict)
                ], ignore_index=True)

datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done


In [9]:
df_filtered = df[df['events'].isin([1, 0])]


game_pk_df = pd.DataFrame(columns=statsapi.schedule(game_id=566083)[0].keys())

venue_dict = {}

#             grabs metadata for respective games such as projected starting pitchers and venue names
for game in df_filtered['game_pk'].unique():
    print(game)
    game_data = statsapi.schedule(game_id=int(game))[-1]
    game_pk_df = game_pk_df.append(game_data, ignore_index=True)
    print(game)

492528
492528
492527
492527
492526
492526
492525
492525
492524
492524
492523
492523
492522
492522
492521
492521
492520
492520
492519
492519
492518
492518
492517
492517
492516
492516
492515
492515
492514
492514
492513
492513
492512
492512
492511
492511
492510
492510
492509
492509
492508
492508
492507
492507
492506
492506
492505
492505
492504
492504
492503
492503
492502
492502
492501
492501
492500
492500
492499
492499
492498
492498
492497
492497
492496
492496
492495
492495
492494
492494
492493
492493
492492
492492
492491
492491
492490
492490
492489
492489
492488
492488
492487
492487
492486
492486
492485
492485
492484
492484
492483
492483
492482
492482
492481
492481
492480
492480
492479
492479
492478
492478
492477
492477
492476
492476
492475
492475
492474
492474
492473
492473
492472
492472
492471
492471
492470
492470
492469
492469
492468
492468
492467
492467
492466
492466
492465
492465
492464
492464
492463
492463
492462
492462
492461
492461
492460
492460
492459
492459
492458
492458
492457

491950
491949
491949
491948
491948
491947
491947
491946
491946
491945
491945
491944
491944
491943
491943
491942
491942
491941
491941
491940
491940
491939
491939
491938
491938
491937
491937
491936
491936
491935
491935
491934
491934
491933
491933
491932
491932
491931
491931
491930
491930
491929
491929
491928
491928
491927
491927
491926
491926
491925
491925
491924
491924
491923
491923
491922
491922
491921
491921
491920
491920
491919
491919
491918
491918
491917
491917
491916
491916
491915
491915
491914
491914
491913
491913
491912
491912
491911
491911
491910
491910
491909
491909
491908
491908
491907
491907
491906
491906
491905
491905
491904
491904
491903
491903
491893
491893
491902
491902
491901
491901
491900
491900
491899
491899
491898
491898
491897
491897
491896
491896
491895
491895
491894
491894
491892
491892
491891
491891
491890
491890
491889
491889
491888
491888
491887
491887
491886
491886
491885
491885
491884
491884
491883
491883
491882
491882
491881
491881
491880
491880
491879
491879

491364
491363
491363
491362
491362
491361
491361
491360
491360
491359
491359
491358
491358
491357
491357
491356
491356
491355
491355
491354
491354
491353
491353
491352
491352
491351
491351
491350
491350
491349
491349
491348
491348
491347
491347
491346
491346
491345
491345
491344
491344
491343
491343
491342
491342
491341
491341
491340
491340
491339
491339
491338
491338
491337
491337
491336
491336
491335
491335
491334
491334
491333
491333
491332
491332
491331
491331
491330
491330
491329
491329
491328
491328
491327
491327
491326
491326
491325
491325
491324
491324
491323
491323
491322
491322
491321
491321
491320
491320
491319
491319
491318
491318
491317
491317
491316
491316
491315
491315
491314
491314
491313
491313
491312
491312
491311
491311
491310
491310
491309
491309
491308
491308
490519
490519
490452
490452
491307
491307
491306
491306
491305
491305
491304
491304
491303
491303
491302
491302
491301
491301
491300
491300
491299
491299
491298
491298
491297
491297
491295
491295
491294
491294

490782
490781
490781
490780
490780
490779
490779
490778
490778
490777
490777
490776
490776
490775
490775
490774
490774
490773
490773
490772
490772
490771
490771
490770
490770
490769
490769
490768
490768
490767
490767
490766
490766
490765
490765
490764
490764
490763
490763
490762
490762
490761
490761
490760
490760
490759
490759
490758
490758
490757
490757
490756
490756
490755
490755
490754
490754
490753
490753
490752
490752
490751
490751
490750
490750
490749
490749
490748
490748
490738
490738
490747
490747
490746
490746
490745
490745
490744
490744
490743
490743
490742
490742
490740
490740
490739
490739
490737
490737
490736
490736
490735
490735
490734
490734
490733
490733
490732
490732
490731
490731
490730
490730
490729
490729
490728
490728
490727
490727
490726
490726
490725
490725
490724
490724
490723
490723
490722
490722
490721
490721
490720
490720
490719
490719
490718
490718
490717
490717
490716
490716
490715
490715
490714
490714
490713
490713
490712
490712
490711
490711
490710
490710

490181
490181
490180
490180
490179
490179
490178
490178
490177
490177
490176
490176
490175
490175
490174
490174
490173
490173
490172
490172
490171
490171
490170
490170
490169
490169
490168
490168
490167
490167
490166
490166
490165
490165
490164
490164
490163
490163
490162
490162
490161
490161
490160
490160
490159
490159
490158
490158
490157
490157
490156
490156
490155
490155
490154
490154
490153
490153
490152
490152
490151
490151
490150
490150
490149
490149
490148
490148
490147
490147
490146
490146
490145
490145
490144
490144
490142
490142
490141
490141
490140
490140
490139
490139
490138
490138
490137
490137
490135
490135
490121
490121
490134
490134
490133
490133
490132
490132
490131
490131
490130
490130
490129
490129
490128
490128
490127
490127
490126
490126
490125
490125
490123
490123
490122
490122
490120
490120
490136
490136
490119
490119
490118
490118
490117
490117
490116
490116
490115
490115
490114
490114
490113
490113
490102
490102
490112
490112
490111
490111
490109
490109
490108

...

632251
632251
632249
632249
632247
632247
632244
632244
632282
632282
632279
632279
632277
632277
632276
632276
632274
632274
632273
632273
632272
632272
632271
632271
632270
632270
632268
632268
632267
632267
632266
632266
632261
632261
632250
632250
632248
632248
632294
632294
632293
632293
632292
632292
632291
632291
632286
632286
632285
632285
632284
632284
632283
632283
632281
632281
632280
632280
632278
632278
632275
632275
632310
632310
632309
632309
632306
632306
632305
632305
632304
632304
632303
632303
632302
632302
632301
632301
632300
632300
632299
632299
632298
632298
632297
632297
632296
632296
632295
632295
632287
632287
632666
632666
632322
632322
632321
632321
632320
632320
632318
632318
632317
632317
632316
632316
632315
632315
632314
632314
632313
632313
632312
632312
632311
632311
632308
632308
632307
632307
632289
632289
632288
632288
632409
632409
632393
632393
632388
632388
632323
632323
632319
632319
632345
632345
632337
632337
632336
632336
632334
632334
632333

632835
632833
632833
632823
632823
632816
632816
632887
632887
632865
632865
632862
632862
632861
632861
632859
632859
632856
632856
632855
632855
632854
632854
632852
632852
632850
632850
632849
632849
632848
632848
632844
632844
632839
632839
632831
632831
633291
633291
632919
632919
632909
632909
632908
632908
632893
632893
632884
632884
632880
632880
632875
632875
632872
632872
632871
632871
632869
632869
632868
632868
632864
632864
632853
632853
632851
632851
632825
632825
633421
633421
632892
632892
632890
632890
632886
632886
632883
632883
632882
632882
632878
632878
632873
632873
632858
632858
632813
632813
632961
632961
632917
632917
632910
632910
632899
632899
632895
632895
632891
632891
632881
632881
632876
632876
632874
632874
632867
632867
632866
632866
632863
632863
632860
632860
632857
632857
632847
632847
632958
632958
632941
632941
632925
632925
632916
632916
632913
632913
632907
632907
632905
632905
632898
632898
632897
632897
632894
632894
632889
632889
632885
632885

633424
633416
633416
633403
633403
633401
633401
633390
633390
633388
633388
633387
633387
633379
633379
633511
633511
633507
633507
633470
633470
633468
633468
633464
633464
633462
633462
633452
633452
633451
633451
633448
633448
633433
633433
633420
633420
633415
633415
633404
633404
633402
633402
633529
633529
633481
633481
633480
633480
633469
633469
633463
633463
633460
633460
633457
633457
633456
633456
633438
633438
633418
633418
633405
633405
633506
633506
633491
633491
633485
633485
633484
633484
633479
633479
633474
633474
633467
633467
633461
633461
633445
633445
633437
633437
633435
633435
633432
633432
633427
633427
633426
633426
633419
633419
633587
633587
633508
633508
633497
633497
633496
633496
633495
633495
633488
633488
633478
633478
633471
633471
633455
633455
633454
633454
633449
633449
633446
633446
633444
633444
633443
633443
634620
634620
633867
633867
633601
633601
633490
633490
633483
633483
633475
633475
633466
633466
633465
633465
633458
633458
633450
633450

634045
634045
634041
634041
634039
634039
634020
634020
634018
634018
634168
634168
634116
634116
634066
634066
634061
634061
634055
634055
634053
634053
634046
634046
634044
634044
634037
634037
634034
634034
634033
634033
634031
634031
634030
634030
634029
634029
634026
634026
634133
634133
634115
634115
634109
634109
634107
634107
634102
634102
634092
634092
634075
634075
634073
634073
634067
634067
634065
634065
634058
634058
634052
634052
634051
634051
634050
634050
634043
634043
634101
634101
634099
634099
634091
634091
634082
634082
634077
634077
634076
634076
634070
634070
634063
634063
634057
634057
634188
634188
634134
634134
634130
634130
634124
634124
634122
634122
634100
634100
634098
634098
634094
634094
634090
634090
634087
634087
634086
634086
634083
634083
634071
634071
634069
634069
634068
634068
634161
634161
634148
634148
634143
634143
634139
634139
634127
634127
634111
634111
634106
634106
634105
634105
634104
634104
634097
634097
634096
634096
634095
634095
634085

634612
634606
634606
634601
634601
634600
634600
634591
634591
634588
634588
634587
634587
634566
634566
634562
634562
634558
634558
634552
634552
634551
634551
634651
634651
634637
634637
634617
634617
634613
634613
634611
634611
634610
634610
634607
634607
634597
634597
634593
634593
634572
634572
634560
634560
634557
634557
634654
634654
634646
634646
634644
634644
634643
634643
634639
634639
634633
634633
634629
634629
634623
634623
634604
634604
634596
634596
634580
634580
634577
634577
634575
634575
634573
634573
634653
634653
634631
634631
634627
634627
634605
634605
634602
634602
634578
634578
634576
634576
634655
634655
634652
634652
634645
634645
634642
634642
634640
634640
634638
634638
634636
634636
634634
634634
634630
634630
634625
634625
634622
634622
634618
634618
634615
634615


In [10]:
1+1

2

In [11]:
#             grabs coordinate date for venues from Parks.csv which needs to be manually updated every time
#             there's a new venue or name change
venue_coords = pd.read_csv('Parks.csv')

for venue in game_pk_df['venue_name'].unique():
    print(venue)
    try:
        city = venue_coords[venue_coords['NAME'] == venue]['CITY'].iloc[0]
        lat = venue_coords[venue_coords['NAME'] == venue]['Latitude'].iloc[0]
        lon = venue_coords[venue_coords['NAME'] == venue]['Longitude'].iloc[0]
        alt = venue_coords[venue_coords['NAME'] == venue]['Altitude'].iloc[0]
        venue_dict[venue] = np.array((city, lat, lon, alt))
    except IndexError:
        raise IndexError(venue)

#             converts game start times from UTC to local times, based on hand written mapping
#             in convert_UTC_to_local for the purposes of collating start times to cut down on
#             Visual Crossing api calls as they cost money after the first 1000 per day
#

game_pk_df['local_datetime'] = game_pk_df.apply(lambda row: convert_UTC_to_local(row), axis=1)
game_pk_df['local_datetime'] = game_pk_df['local_datetime'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))

datetime_coordinate_matching = {}
for index, row in game_pk_df.iterrows():
    city, lat, lon, alt = venue_dict[row['venue_name']]
    dc_datetime = row['local_datetime']
    if dc_datetime in datetime_coordinate_matching.keys():
        datetime_coordinate_matching[dc_datetime].append(','.join([str(lat), str(lon)]))
    else:
        datetime_coordinate_matching[dc_datetime] = [','.join([str(lat), str(lon)])]

game_pk_df['coordinates'] = game_pk_df['venue_name'].apply(lambda x: ','.join(venue_dict[x][1:3]))
game_pk_df['alt'] = game_pk_df['venue_name'].apply(lambda x: venue_dict[x][3])

#             calls Visual Crossing for historical weather data for each game

Yankee Stadium
Angel Stadium of Anaheim
AT&T Park
Nationals Park
Globe Life Park in Arlington
Citizens Bank Park
Busch Stadium
Coors Field
Fenway Park
Target Field
Wrigley Field
Progressive Field
Tropicana Field
Marlins Park
Kauffman Stadium
Guaranteed Rate Field
Miller Park
Oakland Coliseum
Chase Field
Dodger Stadium
PNC Park
Citi Field
Oriole Park at Camden Yards
SunTrust Park
Rogers Centre
Comerica Park
Minute Maid Park
Petco Park
Safeco Field
Great American Ball Park
BB&T Ballpark
Angel Stadium
Estadio de Beisbol Monterrey
Hiram Bithorn Stadium
Oracle Park
T-Mobile Park
London Stadium
TD Ameritrade Park
Tokyo Dome
Globe Life Field
Sahlen Field
Truist Park
loanDepot park
American Family Field
Field of Dreams
TD Ballpark


In [12]:
datetime_coordinate_matching

{'2017-10-01T15:05:00': ['40.829586,-73.926413',
  '38.872987,-77.007435',
  '39.906109,-75.166485',
  '42.346561,-71.097337'],
 '2017-10-01T12:07:00': ['33.80029,-117.882685'],
 '2017-10-01T12:05:00': ['37.778324,-122.389221'],
 '2017-10-01T14:05:00': ['32.751164,-97.082546'],
 '2017-10-01T14:15:00': ['38.622622,-90.192841', '39.051604,-94.480149'],
 '2017-10-01T13:10:00': ['39.756175,-104.99413'],
 '2017-10-01T14:10:00': ['44.981749,-93.278026'],
 '2017-10-01T14:20:00': ['41.948314,-87.655397'],
 '2017-10-01T15:10:00': ['41.496005,-81.685326',
  '27.768254,-82.653431',
  '25.778165,-80.219541'],
 '2017-09-30T13:05:00': ['40.829586,-73.926413',
  '37.778324,-122.389221',
  '42.346561,-71.097337'],
 '2017-09-30T18:07:00': ['33.80029,-117.882685'],
 '2017-09-30T19:05:00': ['38.872987,-77.007435',
  '32.751164,-97.082546',
  '39.906109,-75.166485'],
 '2017-09-30T15:15:00': ['38.622622,-90.192841'],
 '2017-09-30T18:10:00': ['39.756175,-104.99413',
  '44.981749,-93.278026',
  '27.768254,-8

In [13]:
# removing weather

# from api_keys import visual_crossing
# import requests

# weather_df = pd.DataFrame()
# for key, value in datetime_coordinate_matching.items():
#     print(key)
#     url_locations = '|'.join(value)
#     URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history' \
#           f'?&aggregateHours=1&startDateTime={key}&endDateTime={key}&unitGroup=us&contentType=json&loc' \
#           f'ation={url_locations}&key={visual_crossing} '
# #     if api_counter == 1000:
# #         print(
# #             'Prepreprocessing will continue at ' + (datetime.now() + timedelta(seconds=86400)).strftime('%H:%M:%S'))
# #         sleep(86400)
# #         api_counter = 0
# #         api_counter += 1
# #         response = requests.get(URL)
# #     else:
#     api_counter += 1
#     response = requests.get(URL)
#     try:
#         data = response.json()
#         locations = list(data['locations'].keys())
#         for each in locations:
#             values = data['locations'][each]['values'][0]

#             # creates a df containing coordinate data and returned weather data to later be added with other
#             # collected data

#             weather_df = weather_df.append({**{'coordinates': each}, **values}, ignore_index=True)
#     except:
#         raise IndexError(key, value)
# #             cuts off the trailing stamp in datetimeStr signifying the time zone (e.g.-05:00)
# weather_df['datetimeStr'] = weather_df['datetimeStr'].apply(lambda x: x[:-6])
# weather_df = weather_df.rename(columns={'datetimeStr': 'local_datetime'})

# weather_df.to_csv('weather_df')

In [14]:
# games_and_weather = pd.merge(
#     game_pk_df,
#     weather_df,
#     how="left",
#     on=None,
#     left_on=['local_datetime', 'coordinates'],
#     right_on=['local_datetime', 'coordinates'],
#     left_index=False,
#     right_index=False,
#     sort=False,
#     suffixes=("_gpk", "_acw"),
#     copy=True,
#     indicator=False,
#     validate=None,
# )

df_detailed = pd.merge(
    game_pk_df,
    df_filtered,
    how="right",
    on=None,
    left_on='game_id',
    right_on='game_pk',
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_gpk", "_dff"),
    copy=True,
    indicator=False,
    validate=None,
)

In [15]:
#             additional useful data added to df
df_detailed['covid_doubleheader'] = df_detailed.apply(
    lambda row: 1 if row['game_year'] in [2020, 2021] and row['doubleheader'] == 'Y' else 0, axis=1)
df_detailed['designated_hitter'] = df_detailed.apply(
    lambda row: 1 if row['home_name'] in american_league_teams or row['game_year'] == 2020 else 0, axis=1)

#         read in entire relevant history (2 years prior to game in data) in order to assemble
#         cumulative stats for players
history = pd.concat([
    set_types(pd.read_csv('data/2017_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2018_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2019_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2020_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2021_statcast_pbp.csv'), type_dict)
], ignore_index=True)

#         pitcher_df combines away and home pitchers into one column
#         each row is a projected starting pitcher and the game they're
#         starting as well as the date
pitcher_df = pd.concat([df_detailed[['game_id', 'game_date_gpk', 'away_probable_pitcher']].rename(
    columns={'away_probable_pitcher': 'probable_pitcher'}),
                        df_detailed[['game_id', 'game_date_gpk', 'home_probable_pitcher']].rename(
                            columns={'home_probable_pitcher': 'probable_pitcher'})])
pitcher_df = pitcher_df.drop_duplicates()


datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done


In [16]:
def derived_cumulative_stats(player, stats_type, history, start_date=None,
                             end_date=datetime.strptime('2021-06-30', '%Y-%m-%d')) -> pd.DataFrame:

    """Takes in a player and returns a dataframe containing the player's stats derived from their play over the prior two years"""

    print(end_date)
    print(player)

    #    statistics meant to describe last 2 years

    if start_date is None:
        start_date = end_date - timedelta(weeks=102)

    #    if player's name is passed in, rather than their mlbam ID, lookup ID

    if type(player) == str:
        if player[-3:] == 'Jr.' or player[-3:] == 'III':
            player = player[:-4]
        player = player.split(' ')
        player.reverse()
        try:
            player_info = pybaseball.playerid_lookup(player[0], player[-1])
            if len(player_info) > 1:
                print(type(player_info))
                for index, row in player_info.iterrows():
                    if row['mlb_played_last'] < end_date.year:
                        pass
                    else:
                        player_info = row
                player = player_info['key_mlbam']
            elif player_info.empty:
                player_info = pybaseball.playerid_lookup(player[0], player[-1], fuzzy=True).iloc[0]
                player = player_info['key_mlbam']
            else:
                try:
                    player = player_info.iloc[0]['key_mlbam']
                except:
                    raise NameError(player_info, player)
        except:
            raise NameError(player)

    history = history[(history['game_date'] >= start_date) & (history['game_date'] <= end_date)]

    history = history[history['events'].isin([1, 0])]

    #     if player is a pitcher, send down derive_pitcher path
    if stats_type == 'pitcher':
        return derive_pitcher(player, history)
    elif stats_type == 'batter':
        return derive_batter(player, history)

def derive_batter(player, history):
    try:
        history = history[history['batter'] == player]
    except:
        raise NameError(player)
    #     H/PA per pitch type
    #     K/PA per pitch type
    #     H/PA vs righties, lefties
    #     PA/G
    #     avg_launch_angle
    #     avg_launch_speed
    #     xBA based on estimated_ba_using_speedangle

    games = history['game_pk'].unique()
    pas_list = []

    batter = {}

    #         find average number of plate appearances per game
    for game in games:
        pas = len(history[history['game_pk'] == game])
        pas_list.append(pas)
    batter['games_played_last_2_years_batter'] = len(games)
    batter['PA/G_batter'] = np.array(pas_list).mean()

    #         righty/lefty split
    pitcher_right = history[history['pitcher_righty'] == 1]
    pas_r = len(pitcher_right)
    hits_r = len(pitcher_right[pitcher_right['events'] == 1])

    batter['H/PA_against_R'] = hits_r / pas_r if pas_r > 0 else 0
    pitcher_left = history[history['pitcher_righty'] == 0]
    pas_l = len(pitcher_left)
    hits_l = len(pitcher_left[pitcher_left['events'] == 1])

    batter['H/PA_against_L'] = hits_l / pas_l if pas_l > 0 else 0
    if pas_r > 0 or pas_l > 0:
        batter['H/PA_batter'] = (hits_r + hits_l) / (pas_r + pas_l)
    else:
        batter['H/PA_batter'] = 0
    batter['avg_launch_angle'] = history['launch_angle'].mean()
    batter['avg_launch_speed'] = history['launch_speed'].mean()
    batter['xBA'] = history['estimated_ba_using_speedangle'].mean()

    return (batter)

def derive_pitcher(player, history):
    try:
        history = history[history['pitcher'] == player]
    except:
        raise NameError(player)
    print('player filtered')
    #   PITCH METRICS
    pitches = {}
    while True:
        try:
            pitches['pitch_hand'] = statsapi.player_stat_data(player)['pitch_hand']
            break
        except:
            print('pitch_hand_error')
            sleep(10)

    #     filter history down to plate appearance ending plays

    games = history['game_pk'].unique()
    plate_appearances_list = []
    num_hits_list = []
    inning_list = []
#     
#     
#     
    print(games)
#     
#     
#     
    for game in games:
        game = history[history['game_pk'] == game]
        inning = game['inning'].max()
        inning_list.append(inning)
        plate_appearances = len(game)
        plate_appearances_list.append(plate_appearances)
        #             for each inning, collect number of hit events to determine average inning
        for i in range(1, inning + 1):
            num_hits = len(game[(game['inning'] == i) & (game['events'] == 1)])
            num_hits_list.append(num_hits)
    pitches['games_played_last_2_years_pitcher'] = len(games)
    pitches['avg_innings_per_appearance'] = np.array(inning_list).mean()
    pitches['avg_PAs_per_appearance_pitcher'] = np.array(plate_appearances_list).mean()
    pitches['avg_hits_per_inning'] = np.array(num_hits_list).mean()

    #         L/R splits
    left_hitter = history[history['batter_righty'] == 0]
    right_hitter = history[history['batter_righty'] == 1]
    l_pas = len(left_hitter)
    r_pas = len(right_hitter)
    l_hits = len(left_hitter[(history['events'] == 1)])
    r_hits = len(right_hitter[(history['events'] == 1)])

    if l_pas > 0 and r_pas > 0:
        pitches['H/PA_pitcher'] = (l_hits + r_hits) / (l_pas + r_pas)
        pitches['against_lefties_H/PA'] = (l_hits) / (l_pas)
        pitches['against_righties_H/PA'] = (r_hits) / (r_pas)
    elif l_pas > 0:
        pitches['H/PA_pitcher'] = (l_hits + r_hits) / (l_pas + r_pas)
        pitches['against_lefties_H/PA'] = (l_hits) / (l_pas)
        pitches['against_righties_H/PA'] = 0
    elif r_pas > 0:
        pitches['H/PA_pitcher'] = (l_hits + r_hits) / (l_pas + r_pas)
        pitches['against_lefties_H/PA'] = 0
        pitches['against_righties_H/PA'] = (r_hits) / (r_pas)
    else:
        pitches['H/PA_pitcher'] = 0
        pitches['against_lefties_H/PA'] = 0
        pitches['against_righties_H/PA'] = 0
    return (pitches)

In [17]:
derived_cumulative_stats('Charlie Morton', 'pitcher', history,
                                end_date=datetime.strptime('2017-09-29', '%Y-%m-%d') - timedelta(days=1))

2017-09-28 00:00:00
Charlie Morton
Gathering player lookup table. This may take a moment.
<class 'pandas.core.frame.DataFrame'>
player filtered
[492409 492307 492220 492111 492042 491980 491888 491825 491752 491678
 491595 491517 491440 491395 490790 490721 490644 490583 490514 490439
 490356 490288 490196 490133]


{'pitch_hand': 'Right',
 'games_played_last_2_years_pitcher': 24,
 'avg_innings_per_appearance': 6.041666666666667,
 'avg_PAs_per_appearance_pitcher': 24.833333333333332,
 'avg_hits_per_inning': 0.8344827586206897,
 'H/PA_pitcher': 0.20302013422818793,
 'against_lefties_H/PA': 0.1510791366906475,
 'against_righties_H/PA': 0.24842767295597484}

In [18]:
pybaseball.playerid_lookup('Morton', 'Charlie')

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,morton,charlie,119424,mortc101,mortoch01,1009231,1882.0,1885.0
1,morton,charlie,450203,mortc002,mortoch02,4676,2008.0,2021.0


In [19]:
pitcher_df[(pitcher_df['avg_innings_per_appearance'].isna()) & (pitcher_df['game_date_gpk'].str.startswith('2017'))]

KeyError: 'avg_innings_per_appearance'

In [ ]:
#         if probable pitcher is unknown, drop row
pitcher_df['probable_pitcher'] = pitcher_df['probable_pitcher'].replace('', np.nan)
pitcher_df = pitcher_df.dropna(subset=['probable_pitcher'])
pitcher_df = pitcher_df.reset_index(drop=True)

#         runs pitcher_df through derived_cumulative_stats function to get their recent history
pitcher_stats = pitcher_df.apply(lambda row: pd.Series(
    derived_cumulative_stats(row['probable_pitcher'], 'pitcher', history,
                                end_date=datetime.strptime(row['game_date_gpk'], '%Y-%m-%d') - timedelta(days=1))),
                                 axis=1)

pitcher_df = pd.merge(
    pitcher_stats,
    pitcher_df,
    how="left",
    on=None,
    left_on=None,
    right_on=None,
    left_index=True,
    right_index=True,
    sort=False,
    suffixes=("_s", "_df"),
    copy=True,
    indicator=False,
    validate=None,
)


In [ ]:
#         re-matches home batters with away pitchers and vice versa

home_batters = df_detailed[df_detailed['home'] == 1]
away_batters = df_detailed[df_detailed['home'] == 0]

df_wpitching_p1 = pd.merge(
    pitcher_df,
    home_batters,
    how="right",
    on=None,
    left_on=['game_id', 'probable_pitcher'],
    right_on=['game_id', 'away_probable_pitcher'],
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_p", "_df"),
    copy=True,
    indicator=False,
    validate=None,
)

df_wpitching_p2 = pd.merge(
    pitcher_df,
    away_batters,
    how="right",
    on=None,
    left_on=['game_id', 'probable_pitcher'],
    right_on=['game_id', 'home_probable_pitcher'],
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_p", "_df"),
    copy=True,
    indicator=False,
    validate=None,
)

df_wpitching = pd.concat([df_wpitching_p1, df_wpitching_p2], ignore_index=True)


In [ ]:
df_wpitching = df_wpitching.sort_values(
    ['game_date_gpk_df', 'game_pk', 'home_name', 'inning', 'home', 'outs_when_up', 'at_bat_number',
     'pitch_number']).reset_index(drop=True)

df_wpitching = df_wpitching.rename(columns={'events': 'got_a_hit'})

#         drops repeat batters in the same game, while preserving the 1 value for got_a_hit, if present
df_wpitching = df_wpitching.sort_values('got_a_hit').drop_duplicates(subset=['game_id', 'batter'],
                                                                     keep='last').sort_values(
    ['game_date_gpk_df', 'game_pk', 'home_name', 'inning', 'home', 'outs_when_up', 'at_bat_number',
     'pitch_number']).reset_index(drop=True)

#     combines batting stats with pitching/statcast/statsapi/weather data

batting_stats = df_wpitching.apply(lambda row: pd.Series(
    derived_cumulative_stats(row['batter'], 'batter', history,
                                end_date=datetime.strptime(row['game_date_gpk_df'], '%Y-%m-%d') - timedelta(days=1))),
                                   axis=1)

In [ ]:
df_final = pd.merge(
    batting_stats,
    df_wpitching,
    how="right",
    on=None,
    left_on=None,
    right_on=None,
    left_index=True,
    right_index=True,
    sort=False,
    suffixes=("_bs", "_df"),
    copy=True,
    indicator=False,
    validate=None,
)
#

In [ ]:
df_final = df_final.rename(columns={'games_played_last_2_years_df': 'games_played_last_2_years_pitcher', 'games_played_last_2_years_bs': 'games_played_last_2_years_batter'})

#     transforms date and datetime columns into easily comparable numbers
df_final['local_datetime'] = pd.to_datetime(df_final['local_datetime'])
df_final['local_date'] = df_final['local_datetime'].apply(lambda x: x.toordinal())
df_final['local_datetime'] = df_final['local_datetime'].apply(lambda x: x.timestamp())

#     drop columns that are functionally duplicates, contain information that wouldn't be known prior to game,
#     or are irrelevant


drop_columns= ['player_name',
 'game_type_dff', 'pitch_type',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'pitcher',
 'events',
 'description',
 'spin_dir',
'spin_rate_deprecated',
 'break_angle_deprecated',
 'break_length_deprecated',
 'zone',
 'des',
  'p_throws',
'type',
 'hit_location',
 'bb_type',
 'balls',
 'strikes',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'hc_x',
 'hc_y',
 'tfs_deprecated',
 'tfs_zulu_deprecated',
  'sv_id',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'pitcher.1',
 'fielder_2.1',
 'fielder_3',
 'fielder_4',
 'fielder_5',
 'fielder_6',
 'fielder_7',
 'fielder_8',
 'fielder_9',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimated_woba_using_speedangle',
 'woba_value',
 'woba_denom',
 'babip_value',
 'iso_value',
 'launch_speed_angle',
 'at_bat_number',
 'pitch_number',
 'pitch_name',
 'home_score_dff',
 'away_score_dff',
 'bat_score',
 'fld_score',
 'post_away_score',
 'post_home_score',
 'post_bat_score',
 'post_fld_score',
 'if_fielding_alignment',
 'of_fielding_alignment',
 'spin_axis',
 'delta_home_win_exp',
 'delta_run_exp',
 'game_pk',
 'game_id','game_date_gpk_p',
 'game_datetime',
 'game_date_gpk_df',
 'game_type_gpk',
 'datetime',
#  'datetimeStr',
 'game_date_dff','winning_team',
 'losing_team',
 'winning_pitcher',
 'losing_pitcher',
 'save_pitcher',
 'summary', 'home_probable_pitcher',
 'away_probable_pitcher',
 'home_pitcher_note',
 'away_pitcher_note',
 'away_score_gpk',
 'home_score_gpk',
 'current_inning',
 'inning_state',
 'venue_id',
 'status',
 'home_team', 'away_team', 'home_id', 'away_id', 'pitcher_righty',
   'game_type',
  'solarenergy', 'solarradiation', 'snow', 'precipcover', 'info',
              'maxt', 'mint', 'weathertype']

for col in df_final.columns:
    if col in drop_columns:
        df_final = df_final.drop(columns=[col])

In [ ]:
df_final.to_csv('df_final.csv')

In [ ]:
def evaluate(estimator, X_train, X_val, y_train, y_val, holdout, roc_auc='proba', output = False):

    X_holdout = holdout.drop(['got_a_hit'], axis=1)
    y_holdout = holdout['got_a_hit']
    
    #     grab predictions
    train_preds = estimator.predict(X_train)
    val_preds = estimator.predict(X_val)
    holdout_preds = estimator.predict(X_holdout)
    
#     output needed for roc_auc score
    if roc_auc == 'skip':
        train_out = False
        val_out = False
        holdout_out = False
    elif roc_auc == 'dec': # not all classifiers have a decision function
        train_out = estimator.decision_function(X_train)
        val_out = estimator.decision_function(X_val)
        holdout_out = estimator.decision_function(X_holdout)
    elif roc_auc == 'proba':
        try:
            train_out = estimator.predict_proba(X_train)[:, 1]
            val_out = estimator.predict_proba(X_val)[:, 1]
            holdout_out = estimator.predict_proba(X_holdout)[:, 1]
        except AttributeError:
            train_out = estimator.predict(X_train)
            val_out = estimator.predict(X_val)
            holdout_out = estimator.predict(X_holdout)

    else:
        raise Exception("The value for roc_auc should be 'skip', 'dec', or 'proba'.")
    
    ac = accuracy_score(y_train, train_preds)
    f1 = f1_score(y_train, train_preds)
    ras = roc_auc_score(y_train, train_out)
    pr = precision_score(y_train, train_preds)
    
    if output == True:
        print('Train Scores')
        print('------------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(train_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')

    ac = accuracy_score(y_val, val_preds)
    f1 = f1_score(y_val, val_preds)
#     print(type(y_val))
    ras = roc_auc_score(y_val, val_out)
    pr = precision_score(y_val, val_preds)
    
    if output == True:
        print('-----------------------------------')
        print('Val Scores')
        print('-----------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(val_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')
    
    ac = accuracy_score(y_holdout, holdout_preds)
    f1 = f1_score(y_holdout, holdout_preds)
#     print(type(holdout['got_a_hit']))
    ras = roc_auc_score(y_holdout, holdout_out)
    pr = precision_score(y_holdout, holdout_preds)
    
    if output == True:
        print('-----------------------------------')
        print('Holdout Scores')
        print('-----------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(holdout_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')

        print('\nVal Data')
        print('-----------')

        plot_confusion_matrix(estimator, X_val, y_val, values_format=',.5g')
        plt.show()
        
        print('Holdout Data')        
        print('-----------')

        plot_confusion_matrix(estimator, X_holdout, y_holdout, values_format=',.5g')
        plt.show()

In [ ]:
def streak_checker(dataframe, output = False, metric = 'odds', ascending=False, combiner=None, timed=False, today=False):
    results = []

    try:
        if 'L/R_split' in metric:
            if combiner == 'add':
                if timed == True:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy = dataframe.copy()
                dataframe_copy['temp'] = dataframe.apply(lambda row: row['H/PA_against_R'] + row['against_lefties_H/PA'] if (row['pitch_hand'] == 'Right' and row['batter_righty'] == 0) else (row['H/PA_against_L'] + row['against_righties_H/PA'] if (row['pitch_hand'] == 'Left' and row['batter_righty'] == '1') else 0), axis=1)
                odds_scaler(dataframe_copy, timed, start)
                dataframe = dataframe_copy
                metric='temp'
            elif combiner == 'multiply':
                if timed == True:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy = dataframe.copy()
                dataframe_copy['temp'] = dataframe.apply(lambda row: row['H/PA_against_R'] * row['against_lefties_H/PA'] if (row['pitch_hand'] == 'Right' and row['batter_righty'] == 0) else (row['H/PA_against_L'] * row['against_righties_H/PA'] if (row['pitch_hand'] == 'Left' and row['batter_righty'] == 1) else 0), axis=1)
                odds_scaler(dataframe_copy, timed, start)
                dataframe = dataframe_copy
                metric='temp'
        if type(metric) == list:
            dataframe_copy = dataframe.copy()
            if combiner == 'add':
                if timed == True:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy['temp'] = dataframe.apply(lambda row: row[metric[0]] + row[metric[1]], axis=1)
                odds_scaler(dataframe_copy, timed, start)
            elif combiner == 'multiply':
                if timed:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy['temp'] = dataframe.apply(lambda row: row[metric[0]] * row[metric[1]], axis=1)
                odds_scaler(dataframe_copy, timed, start)
            else:
                raise KeyError(f'{combiner} combiners have not been implemented yet')
            dataframe = dataframe_copy
            metric = 'temp'
    except KeyError:
        raise KeyError("metric must be chosen from dataframe's columns")

    date_list = np.sort(dataframe['local_date'].unique())

    for day in date_list:
        best_bet = dataframe[dataframe['local_date'] == day].sort_values(metric, ascending=ascending).iloc[0]
#         return(dataframe[dataframe['local_date'] == day].sort_values(metric, ascending=ascending))
        odds = best_bet[metric]
        date = datetime.fromordinal(best_bet['local_date']).strftime("%b %d %Y")

        player_info = pybaseball.playerid_reverse_lookup([best_bet['batter']])
#         print(best_bet['batter'], 'poop')
        if len(player_info) >1:
            player_info = player_info.iloc[0]
            player = player_info['name_first'] + ' ' + player_info['name_last']
        else:
            player = player_info['name_first'] + ' ' + player_info['name_last']
        if best_bet['home'] == 0:
            team = best_bet['away_name']
        else:
            team = best_bet['home_name']
    #     team = statsapi.lookup_team(player['currentTeam']['id'])[0]['name']
    #     player_name = player['fullName']
        if today:
#             actual_result = None
            return player.title() + ' - ' + team
        else:
            actual_result = best_bet['got_a_hit']
        results.append(actual_result)
        if output == True and len(date_list) > 20 and day > date_list[-20]:
            print(date, player, team, odds, actual_result)
#                 print(date, player, team, odds)
    longest = 0
    current = 0
    for num in results:
        if num == 1:
            current += 1
        else:
            longest = max(longest, current)
            current = 0
    if output == True and today == False:
        print(f'Longest streak in set ({len(results)} days): ', longest)
        print(f'Total correct guesses in set ({len(results)} days): ', sum(results), sum(results)/len(results))

def odds_scaler(dataframe_copy, timed, start):
    mms = MinMaxScaler()
    dataframe_copy['temp'] = pd.DataFrame(mms.fit_transform(dataframe_copy[['temp']]))

    if timed:
        print(datetime.now() - start)

In [ ]:
def run_model(clf, X_train, X_val, y_train, y_val, holdout):

    probability = True
    
    clf.fit(X_train.head(100), y_train.head(100))

    evaluate(clf, X_train.head(100), X_val.head(100), y_train.head(100), y_val.head(100), holdout);

    runtime = datetime.now() - start

    if runtime > timedelta(seconds=5):
        print(f'Model would take {runtime.total_seconds()*200/60} minutes to run.')
    else:
        print('Model should take between {:.0f} and {:.0f} minutes to run and finish by {}'.format(runtime.total_seconds()*20/60, runtime.total_seconds()*200/60, (datetime.now()+(timedelta(seconds=(runtime.total_seconds()*200)))).strftime("%H:%M")))
        clf.fit(X_train, y_train)
        
        with open('models/'+list(clf.named_steps.keys())[1], 'wb') as f:
            pickle.dump(clf, f, protocol=pickle.HIGHEST_PROTOCOL)
            
        evaluate(clf, X_train, X_val, y_train, y_val, holdout, output=True)
        plot_roc_curve(clf, X_val, y_val)
        plt.show()
        
        try:
            val_df_odds = pd.Series(clf.predict_proba(X_val)[:, 1], name='odds')
            holdout_df_odds = pd.Series(clf.predict_proba(holdout.drop(['got_a_hit'], axis=1))[:, 1], name='odds')
        except AttributeError:
            probability = False
            val_df_odds = pd.Series(clf.predict(X_val), name= 'odds')
            holdout_df_odds = pd.Series(clf.predict(holdout.drop(['got_a_hit'], axis=1)), name='odds')

        val_df = X_val.assign(got_a_hit = y_val).reset_index(drop=True)
        val_df = val_df.assign(odds=val_df_odds)
        holdout = holdout.reset_index(drop=True)
        holdout = holdout.assign(odds=holdout_df_odds)
#         return(holdout)

# This check is to prevent returning recommendations based solely on models that do not return probabilities. However, it appears that randomly selected 1s may perform better than sorted 1s once passed through some filter such as plate appearances per game

#         TODO change to .issubset()
#         if set(val_df_odds.value_counts().index) == set([1, 0, 1.0, 0.0]):
#             print(str(clf.get_params()['steps'][1][1]), "doesn't return probabilities, so no streak results will be returned based on this alone.")
#             pass
#         else:
#             print()
        print('-----------------------------------')

        print('Val data')
        print('-----------')

        streak_checker(val_df, output=True)
        print('Holdout data')
        print('-----------')

        streak_checker(holdout, output=True)
            
        print('Odds multiplied by PA/G_batter')
        print('-----------')
        
        print('Val data')
        print('-----------')
        streak_checker(val_df, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply')
        print('Holdout data')
        print('-----------')
        streak_checker(holdout, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply')
        print('-----------')
        print('Odds added to PA/G_batter')
        print('-----------')

        print('Val data')
        print('-----------')
        streak_checker(val_df, output=True, metric=['L/R_split', 'odds'], combiner='add')
        print('-----------')
        print('Holdout data')
        print('-----------')
        streak_checker(holdout, output=True, metric=['L/R_split', 'odds'], combiner='add')

In [ ]:
def train_test_split(df, test_size=.25):
    
    '''My simplified train_test_split which just cuts the dataframe by date, in order to accurately evaluate streak results, to roughly the test size requested'''
    
    sample_size = len(df)
    
    train_size = 1 - test_size
    
    train_size = round(train_size * sample_size)
    
    df = df.sort_values(['local_date']).reset_index(drop=True)
    
    date_cutoff = df.iloc[train_size]['local_date']

    train = df[df['local_date'] <= date_cutoff]
    val = df[df['local_date'] > date_cutoff]
    
    X_train = train.drop(['got_a_hit'], axis=1)
    y_train = train['got_a_hit']
    
    X_val = val.drop(['got_a_hit'], axis=1)
    y_val = val['got_a_hit']
    
    return(X_train, X_val, y_train, y_val)

In [ ]:
df_final = pd.read_csv('df_final.csv', index_col=0)

In [ ]:
df_final.columns.to_list()

In [ ]:
df_final

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_final)

In [ ]:
num_cols = [] # all numeric columns
cols_to_ohe = [] # doubleheader, conditions, batter_righty
cols_to_targ = [] # all other object columns

for c in X_train.columns:
    if X_train[c].dtype in ['float64', 'int64'] and c not in ['game_num', 'batter', 'fielder_2', 'umpire']:
        num_cols.append(c)
    elif len(X_train[c].unique()) < 10:
        cols_to_ohe.append(c)
    else:
        cols_to_targ.append(c)

In [ ]:
num_transformer = Pipeline(steps=[
    ('num_imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

ohe_transformer = Pipeline(steps=[
    ('ohe_imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('ohencoder', OneHotEncoder(handle_unknown='ignore'))
])

target_encoder = Pipeline(steps=[
    ('freq_enc', ce.target_encoder.TargetEncoder()),
    ('freq_imputer', SimpleImputer(strategy='constant', fill_value=0))
    
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('ohe', ohe_transformer, cols_to_ohe),
        ('target', target_encoder, cols_to_targ)
    ])

# Baseline - dummy models
Pick player with the highest Plate Appearances per Game over the last 2 years

In [ ]:
streak_checker(df_final, output=True, metric='PA/G_batter')
streak_checker(test_df, output=True, metric='PA/G_batter')

Observations - this seems to trend towards a small group of players except for when a player makes their mLB debut towards the top of the lineup (Wander Franco, Greg Deichmann). Sample size is an issue - could be addressed by either looking at games played or excluding PA/G above 4.9.



Pick player with the highest launch speed

In [ ]:
streak_checker(df, output=True, metric='avg_launch_speed')
streak_checker(test_df, output=True, metric='avg_launch_speed')

The metrics point to this being mostly useless which is backed up by the fact that it seems to occasionally choose pitchers (Hunter Strickland, Zack Greinke), when it's not fawning over Aaron Judge. Could be worth excluding pitchers as a class though would be hard to implement with the current dataset, not to mention the fact that 2-way players would need to be excluded from that filter.

In [ ]:
streak_checker(df, output=True, metric=['avg_launch_angle', 'avg_launch_speed'], combiner='add')
streak_checker(test_df, output=True, metric=['avg_launch_angle', 'avg_launch_speed'], combiner='add')

Exaserbates bias towards small sample sizes, leading part time players to dominate selection rather than proven hitters

Would also be worth checking lefty righty splits

In [ ]:
streak_checker(df, output=True, metric='L/R_split', combiner='add')
streak_checker(test_df, output=True, metric='L/R_split', combiner='add')
streak_checker(df, output=True, metric='L/R_split', combiner='multiply')
streak_checker(test_df, output=True, metric='L/R_split', combiner='multiply')

Nothing stands out in these groups. Seems to be a mix of journeymen and all-stars.

# Model 1 - Decision Tree

In [ ]:
start = datetime.now()
dt = DecisionTreeClassifier()

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('tree', dt)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 2: Logistic Regression

In [ ]:
start = datetime.now()

logreg = LogisticRegression(solver='sag')

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('logreg', logreg)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 3: SVC

In [ ]:
start = datetime.now()

svc = SVC()


clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svc', svc)
])


run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 3.5: LinearSVC

In [ ]:
start = datetime.now()

Linearsvc = LinearSVC()


clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('linearsvc', Linearsvc)
])


run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 4: KNN

In [ ]:
start = datetime.now()

knn = KNeighborsClassifier(n_jobs=-1)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', knn)
])


run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 5: Random Forest

In [ ]:
start = datetime.now()

rfc = RandomForestClassifier(n_jobs=-1)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rfc', rfc)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 6: AdaBoost


In [ ]:
start = datetime.now()

ada = AdaBoostClassifier(random_state=42)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('ada', ada)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 7: GradientBoost


In [ ]:
start = datetime.now()

gbm = GradientBoostingClassifier()
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('gbm', gbm)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 8: XGBoost


In [ ]:
start = datetime.now()

xbg = xgb.XGBClassifier(random_state=42)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xbg', xbg)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 9: Random Forest with Grid Search


In [ ]:
start = datetime.now()

parameters = {
    'min_samples_split': [3, 5, 100], 
    'n_estimators' : [100, 300],
        'max_depth': [3, 5, 15, 25],

}

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Daily Guess

In [ ]:
def time_date_weather_data(game_pk_df):
    venue_coords = pd.read_csv('Parks.csv')
    data_collection = {}

    game_pk_df['coordinates'] = game_pk_df['venue_name'].apply(lambda x: ','.join([str(venue_coords[venue_coords['NAME'] == x]['Latitude'].iloc[0]), str(venue_coords[venue_coords['NAME'] == x]['Longitude'].iloc[0])]))
    game_pk_df['alt'] = game_pk_df['venue_name'].apply(lambda x: venue_coords[venue_coords['NAME'] == x]['Altitude'].iloc[0])


    game_pk_df = game_pk_df.sort_values(['game_datetime'])



    game_pk_df['local_datetime'] = game_pk_df.apply(lambda row: convert_UTC_to_local(row), axis=1)
    game_pk_df['local_datetime'] = game_pk_df['local_datetime'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))


    from api_keys import visual_crossing
    import requests

    weather_df = pd.DataFrame()
    for index, row in game_pk_df.iterrows():
#         check if key is past or future
        if datetime.strptime(row['game_datetime'], "%Y-%m-%dT%H:%M:%SZ") > datetime.utcnow():
            URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/forecast?locations={row["coordinates"]}&aggregateHours=1&unitGroup=us&forecastDays=1&shortColumnNames=true&contentType=json&key={visual_crossing}'

        else:
            URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=1&startDateTime={row["local_datetime"]}&endDateTime={row["local_datetime"]}&unitGroup=us&contentType=json&location={row["coordinates"]}&key={visual_crossing}'
        global api_counter
        if api_counter >= 1000:
            print('Prepreprocessing will continue at ' + (datetime.now()+timedelta(seconds = 86400)).strftime('%H:%M:%S'))
            sleep(86400)
            api_counter = 0
            api_counter += 1
            response = requests.get(URL)
        try:
            data = response.json()
            if (
                data['locations'][row['coordinates']]['values'][0]['temp']
                is None
            ):
                URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/forecast?locations={row["coordinates"]}&aggregateHours=1&unitGroup=us&forecastDays=1&shortColumnNames=true&contentType=json&key={visual_crossing}'
                api_counter += 1
                response = requests.get(URL)
                data = response.json()

            if len(data['locations'][row['coordinates']]['values']) > 1:
                for forecast in data['locations'][row['coordinates']]['values']:

                    if forecast['datetimeStr'][:13] == row['local_datetime'][:13]:
                        values = forecast
                        values['datetimeStr'] = row['local_datetime']


            else:
                values = data['locations'][row['coordinates']]['values'][0]
                values['datetimeStr'] = row['local_datetime']
            data_collection[URL] = data
            weather_df = weather_df.append({**{'coordinates': row['coordinates']}, **values}, ignore_index=True)

        except:
            raise IndexError(index, row, response, response.json())
    weather_df = weather_df.rename(columns={'datetimeStr': 'local_datetime'})


    return pd.merge(
        game_pk_df,
        weather_df,
        how="left",
        on=None,
        left_on=['local_datetime', 'coordinates'],
        right_on=['local_datetime', 'coordinates'],
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_gpk", "_wdf"),
        copy=True,
        indicator=False,
        validate=None,
    )

In [ ]:
def lookup_mlb():
    """Get list of MLB team IDs for the purposes of pulling active rosters"""
    params = {
        "activeStatus": 'Y',
        "season": 2021,
        "sportIds": 1,
        "fields": "teams,id,name,league",
    }

    r= statsapi.get('teams', params)

    return {
        team['name']: team['id']
        for team in r['teams']
        if team['league']['id'] in [103, 104]
    }

def roster(teamId, rosterType=None, season=datetime.now().year, date=None):
    """Get the roster for a given team."""
    if not rosterType:
        rosterType = "active"

    params = {"rosterType": rosterType, "season": season, "teamId": teamId}
    if date:
        params.update({"date": date})

    r = statsapi.get("team_roster", params)

    return [x["person"]["id"] for x in r["roster"]]

def lookup_team(lookup_value, activeStatus="Y", season=datetime.now().year, sportIds=1):
    """Get a info about a team or teams based on the team name, city, abbreviation, or file code."""
    params = {
        "activeStatus": activeStatus,
        "season": season,
        "sportIds": sportIds,
        "fields": "teams,id,name,teamCode,fileCode,teamName,locationName,shortName,league",
    }
    r = statsapi.get("teams", params)

    teams = []
    for team in r["teams"]:
        for v in team.values():
            if str(lookup_value).lower() in str(v).lower():
                teams.append(team)
                break

    return (teams)

In [ ]:
# to be run daily
def daily_guess():
    '''Implementing a daily recommendation engine based on the LinearSCV model'''
    # get game data, then weather data, then pitcher data, then batter data

    todays_games = pd.DataFrame()


    for game in statsapi.schedule():
    #     grab 'away_probable_pitcher','home_probable_pitcher','away_name',
    #     'home_name','doubleheader','game_num','venue_name','game_datetime'

    #     df['local_datetime'] = pd.to_datetime(df['local_datetime'])
    #     df['local_date'] = df['local_datetime'].apply(lambda x: x.toordinal())
    #     df['local_datetime'] = df['local_datetime'].apply(lambda x: x.timestamp())

        row = {
            'game_id': game['game_id'],
            'away_probable_pitcher': game['away_probable_pitcher'],
            'home_probable_pitcher': game['home_probable_pitcher'],
            'away_name': game['away_name'],
            'home_name': game['home_name'],
            'doubleheader': game['doubleheader'],
            'game_num': game['game_num'],
            'venue_name': game['venue_name'],
            'game_datetime': game['game_datetime']
        }

        todays_games = todays_games.append(row, ignore_index=True)

    todays_games = todays_games.replace('', np.nan)    
    global games_and_weather
    games_and_weather = time_date_weather_data(todays_games)
# def daily_guess(games_and_weather):
    for i in ['home_probable_pitcher', 'away_probable_pitcher']:
        games_and_weather[i] = games_and_weather[i].fillna('Unknown')
    away_pitchers = games_and_weather['away_probable_pitcher']
    home_pitchers = games_and_weather['home_probable_pitcher']
    global pitchers_stats
    pitchers_stats = pd.DataFrame()

    history = pd.concat([
    set_types(pd.read_csv('data/2019_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2020_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2021_statcast_pbp.csv'), type_dict)
    ], ignore_index=True)
    
    sample = pd.read_csv('df_final', index_col=0)
    pitchers_stats_cols = ['pitch_hand', 'games_played_last_2_years_pitcher',
       'avg_innings_per_appearance', 'avg_PAs_per_appearance_pitcher',
       'avg_hits_per_inning', 'H/PA_pitcher', 'against_lefties_H/PA',
       'against_righties_H/PA', 'probable_pitcher']
    
    average_pitcher = {}

    for col in pitchers_stats_cols:
        if col == 'probable_pitcher':
            col_val = 'Unknown'
        else:
            try:
                col_val = sample[col].mean()
            except TypeError:
                col_val = list(sample[col].value_counts().index)[0]
        average_pitcher[col] = col_val
    
    
    for a in away_pitchers:
        print(a)
        if a == 'Unknown':
            pitchers_stats = pitchers_stats.append(pd.Series(average_pitcher), ignore_index=True)
        else:
            pitchers_stats = pitchers_stats.append({**derived_cumulative_stats(a, 'pitcher', history, end_date=(datetime.now()-timedelta(seconds=86400))), **{'probable_pitcher': a}}, ignore_index=True)

    for h in home_pitchers:
        print(h)
        if h == 'Unknown':
            pitchers_stats = pitchers_stats.append(pd.Series(average_pitcher), ignore_index=True)
        else:
            pitchers_stats = pitchers_stats.append({**derived_cumulative_stats(h, 'pitcher', history, end_date=(datetime.now()-timedelta(seconds=86400))), **{'probable_pitcher': h}}, ignore_index=True)

    game_columns = ['away_name',
                     'home_name',
                     'doubleheader',
                     'game_num',
                     'venue_name',
                     'local_datetime',
                     'coordinates',
                     'cloudcover',
                     'conditions',
                     'dew',
                     'heatindex',
                     'humidity',
                     'precip',
                     'sealevelpressure',
                     'snowdepth',
                     'temp',
                     'visibility',
                     'wdir',
                     'wgust',
                     'windchill',
                     'wspd',
                     'alt'
     ]
            
    batting_stats = pd.DataFrame()
    for index, game in games_and_weather.iterrows():
        print(game[['away_name', 'home_name']])
    #     home batters, away pitcher
        batting_team_id = lookup_mlb()[game['home_name']]
        batting_roster = roster(batting_team_id)
        for b in batting_roster:
            start = datetime.now()
            
            batter_righty = 1 if statsapi.player_stat_data(b)['bat_side'] == 'Right' else 0
            home = 1 if game['home_name'] == statsapi.player_stat_data(b)['current_team'] else 0
            batting_stats = batting_stats.append({**derived_cumulative_stats(b, 'batter', history, end_date=(datetime.now()-timedelta(seconds=86400))),**{'batter': b, 'game_year': datetime.now().year, 'batter_righty': batter_righty, 'home': home}, **pitchers_stats[pitchers_stats['probable_pitcher'] == game['away_probable_pitcher']].iloc[0], **game[game_columns]}, ignore_index=True)
            print(datetime.now()-start)
        #     away batters, home pitcher
        batting_team_id = lookup_mlb()[game['away_name']]
        batting_roster = roster(batting_team_id)
        for b in batting_roster:
            batter_righty = 1 if statsapi.player_stat_data(b)['bat_side'] == 'Right' else 0
            home = 1 if game['home_name'] == statsapi.player_stat_data(b)['current_team'] else 0
            batting_stats = batting_stats.append({**derived_cumulative_stats(b, 'batter', history, end_date=(datetime.now()-timedelta(seconds=86400))), **{'batter': b, 'game_year': datetime.now().year, 'batter_righty': batter_righty, 'home': home}, **pitchers_stats[pitchers_stats['probable_pitcher'] == game['home_probable_pitcher']].iloc[0], **game[game_columns]}, ignore_index=True)
            
    batting_stats['covid_doubleheader'] = batting_stats.apply(
    lambda row: 1 if row['game_year'] in [2020, 2021] and row['doubleheader'] == 'Y' else 0, axis=1)
    
    american_league_teams = np.array(['Boston Red Sox', 'Houston Astros', 'Chicago White Sox', 'Tampa Bay Rays', 'Oakland Athletics', 'Seattle Mariners', 'New York Yankees', 'Toronto Blue Jays', 'Los Angeles Angels', 'Cleveland Indians', 'Detroit Tigers', 'Kansas City Royals', 'Minnesota Twins', 'Texas Rangers', 'Baltimore Orioles'])
    
    batting_stats['designated_hitter'] = batting_stats.apply(
    lambda row: 1 if row['home_name'] in american_league_teams or row['game_year'] == 2020 else 0, axis=1)
    
        #     transforms date and datetime columns into easily comparable numbers
    batting_stats['local_datetime'] = pd.to_datetime(batting_stats['local_datetime'])
    batting_stats['local_date'] = batting_stats['local_datetime'].apply(lambda x: x.toordinal())
    batting_stats['local_datetime'] = batting_stats['local_datetime'].apply(lambda x: x.timestamp())
    
    return(batting_stats)
# TODO
# RUN batting_stats through predictors

In [ ]:
def run_model_todays_data(batting_stats):
    recommendations = []
    for model in os.listdir('models/'):
        with open('models/' + model, 'rb') as f:
            clf = pickle.load(f)
            try:
                todays_odds = pd.Series(clf.predict_proba(batting_stats[X_train.columns])[:, 1], name='odds')
            except AttributeError:
                todays_odds = pd.Series(clf.predict(batting_stats[X_train.columns]), name='odds')

            batting_stats_temp = batting_stats.reset_index(drop=True)
            batting_stats_temp = batting_stats.assign(odds=todays_odds)
            
#             print('-----------------------------------')

#             print(model)
#             print('-----------')

            recommendations.append(streak_checker(batting_stats_temp, output=True, today=True))
#             print(recommendations)
#             print('Holdout data')
#             print('-----------')

#             streak_checker(holdout, output=True)
#             ('-----------------------------------')
#             print('\n\nOdds multiplied by PA/G_batter\n\n')
            
#             print('-----------')

#             print('Val data')
#             print('-----------')
            recommendations.append(streak_checker(batting_stats_temp, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply', today=True))
#             print('Holdout data')
#             print('-----------')
#             streak_checker(holdout, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply')
#             print('-----------')
#             ('-----------------------------------')
#             print('\n\nOdds added to PA/G_batter\n\n')
            
#             print('-----------')

#             print('Val data')
#             print('-----------')
            recommendations.append(streak_checker(batting_stats_temp, output=True, metric=['L/R_split', 'odds'], combiner='add', today=True))
#             print('-----------')
#             print('Holdout data')
#             print('-----------')
#             streak_checker(holdout, output=True, metric=['L/R_split', 'odds'], combiner='add')
    return(recommendations)

In [ ]:
start = datetime.now()

batting_stats = daily_guess()

In [ ]:
recommendations = run_model_todays_data(batting_stats)

from collections import Counter
Counter(recommendations)

In [ ]:
datetime.now() - start

## Daily Runtimes

9/11/21 - 564 seconds